<a href="https://colab.research.google.com/github/CoderBabu143/CoderBabu143/blob/main/MULTICLASS_FLOWERS_USING_VGG16_WITH_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

In [2]:
dataset,info=tfds.load('tf_flowers',with_info=True,as_supervised=True)

In [3]:
#supervised_keys=('image','label')
class_names=info.features['label'].names
class_names

['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

In [4]:
for i,example in enumerate(dataset['train']):
  image,label=example
  save_dir='tf_flowers/train/{}'.format(class_names[label])
  os.makedirs(save_dir, exist_ok=True)
  filepath=save_dir + "/" + "{}_{}.jpg".format(class_names[label],i)
  tf.keras.preprocessing.image.save_img(filepath,image.numpy())
  #print(filepath)
  #break

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential

In [10]:
datagen = ImageDataGenerator(rescale=1/255, validation_split=0.2,
                             rotation_range=10, width_shift_range = 0.1, height_shift_range=0.1,
                             shear_range=0.1, zoom_range=0.1, horizontal_flip=True)

train_generator = datagen.flow_from_directory('tf_flowers/train', target_size=(224,224), batch_size=4,
                                              class_mode="categorical", subset='training')

validation_generator = datagen.flow_from_directory('tf_flowers/train', target_size=(224,224), batch_size=4,
                                              class_mode="categorical", subset='validation')





Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [12]:
# create base model
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(5, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 222, 222, 64)      1792      
                                                                 
 conv2d_9 (Conv2D)           (None, 220, 220, 64)      36928     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 110, 110, 64)      0         
 g2D)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 110, 110, 64)      0         
                                                                 
 conv2d_10 (Conv2D)          (None, 108, 108, 128)     73856     
                                                                 
 conv2d_11 (Conv2D)          (None, 106, 106, 128)     147584    
                                                      

In [14]:
history=model.fit(train_generator,epochs=5,validation_data=validation_generator)

Epoch 1/5
735/735 [==============================] - 88s 111ms/step - loss: 1.3143 - accuracy: 0.4389 - val_loss: 1.1867 - val_accuracy: 0.5226
Epoch 2/5
735/735 [==============================] - 83s 113ms/step - loss: 1.1174 - accuracy: 0.5515 - val_loss: 1.0866 - val_accuracy: 0.6088
Epoch 3/5
735/735 [==============================] - 79s 108ms/step - loss: 1.0064 - accuracy: 0.6131 - val_loss: 0.9859 - val_accuracy: 0.6033
Epoch 4/5
735/735 [==============================] - 79s 108ms/step - loss: 0.9338 - accuracy: 0.6523 - val_loss: 0.9013 - val_accuracy: 0.6484
Epoch 5/5
735/735 [==============================] - 80s 108ms/step - loss: 0.8650 - accuracy: 0.6676 - val_loss: 0.9188 - val_accuracy: 0.6607


In [15]:
# VGG 16 Model
from tensorflow.keras.applications import VGG16
model_vgg16 = VGG16(input_shape=(224,224,3), include_top=False, weights='imagenet')


58889256/58889256 [==============================] - 3s 0us/step


In [16]:
for layer in model_vgg16.layers:
  layer.trainable=False

model_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
model = Sequential()
model.add(model_vgg16)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 25088)             0         
                                                                 
 dense_6 (Dense)             (None, 512)               12845568  
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 27562821 (105.14 MB)
Trainable params: 12848133 (49.01 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history=model.fit(train_generator, epochs=5, validation_data=validation_generator)

Epoch 1/5
735/735 [==============================] - 62s 81ms/step - loss: 1.5503 - accuracy: 0.5332 - val_loss: 0.8228 - val_accuracy: 0.6826
Epoch 2/5
735/735 [==============================] - 59s 80ms/step - loss: 0.9558 - accuracy: 0.6339 - val_loss: 0.6708 - val_accuracy: 0.7524
Epoch 3/5
735/735 [==============================] - 58s 80ms/step - loss: 0.8651 - accuracy: 0.6700 - val_loss: 0.6529 - val_accuracy: 0.7633
Epoch 4/5
735/735 [==============================] - 68s 92ms/step - loss: 0.8183 - accuracy: 0.6890 - val_loss: 0.6348 - val_accuracy: 0.7688
Epoch 5/5
735/735 [==============================] - 59s 81ms/step - loss: 0.7672 - accuracy: 0.7033 - val_loss: 0.6217 - val_accuracy: 0.7565


In [19]:
model.save('flowers_classifier.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
flowers = tf.keras.models.load_model('flowers_classifier.h5')

In [21]:
flowers.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 25088)             0         
                                                                 
 dense_6 (Dense)             (None, 512)               12845568  
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 27562821 (105.14 MB)
Trainable params: 12848133 (49.01 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [22]:
import cv2

img = cv2.imread('/content/tf_flowers/train/roses/roses_1054.jpg')
img = cv2.resize(img, (224,224))
img = np.reshape(img, [1,224,224,3])
img = img/255

preds = flowers.predict(img)

max_idx = np.argmax(preds)
class_indices = {v:k for k,v in train_generator.class_indices.items()}
class_indices[max_idx]

1/1 [==============================] - 1s 733ms/step


'roses'